In [ ]:

from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


# Open the RedBus TSRTC booking page
driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile')
driver.maximize_window()
actions = ActionChains(driver)


# Function to scrape bus names and links
def scrape_current_page():
    time.sleep(3)

    # Simulate scrolling with ActionChains
    footer = driver.find_element(By.CLASS_NAME, 'DC_117_paginationTable')
    head = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]')

    # Scroll down multiple times to ensure all buses are loaded
    for _ in range(2):  # Adjust the range as necessary depending on the number of buses
        actions.move_to_element(head).perform()
        time.sleep(2)
        actions.move_to_element(footer).perform()
        time.sleep(2)

    # Extract bus names and links 
    buses = driver.find_elements(By.CLASS_NAME, 'route')

    # Store bus names and links
    bus_info = []
    for bus in buses:
        bus_name = bus.text
        bus_link = bus.get_attribute('href')
        bus_info.append((bus_name, bus_link))

    return bus_info

# Function to scrape bus details from a specific route page
def scrape_bus_details(driver, route_link, route_name):
    try:
        driver.get(route_link)
        time.sleep(5) 

        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)  # Wait for buses to load

            # Scroll down to load all bus items
            for i in range(30):  # Adjust the range as necessary depending on the number of buses
                actions.send_keys(Keys.PAGE_DOWN).perform()
                time.sleep(1) # Wait for the page to load more content

            actions.send_keys(Keys.END).perform()

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Bus_Routes_Name": route_name,
                    "Bus_Routes_link": route_link,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type_(Sleeper/Seater/AC/Non-AC)": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details

        except Exception as e:
            print(f"Error occurred while scraping bus details for {route_link}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {route_link}: {str(e)}")
        return []

# List to store bus data across all pages
all_bus_info = []

# Scrape data for each page (adjust based on the actual number of pages)
for page_num in range(1, 4):  # Adjust based on the actual number of pages
    print(f"Scraping page {page_num}...")
    bus_data = scrape_current_page()
    all_bus_info.extend(bus_data)
    
    if page_num <= 5:
        try:
            # Find and click the "Next" button
            next_page_button = driver.find_element(By.XPATH, f"//div[@class='DC_117_paginationTable']//div[contains(text(), '{page_num + 1}')]")
            next_page_button.click()
            time.sleep(3)  # Wait for the next page to load
        except Exception as e:
            print(f"Error moving to the next page: {e}")
            break

# Now scrape the details for each bus name and link
all_bus_details = []

for name, link in all_bus_info:
    #print(f"Scraping details for Bus Name: {name}, Link: {link}")
    bus_details = scrape_bus_details(driver, link, name)
    all_bus_details.extend(bus_details)

# Close the browser
driver.quit()

In [3]:
import pandas as pd

df = pd.DataFrame(all_bus_details)
df

,Bus_Routes_Name,Bus_Routes_link,Bus_Name,Bus_Type_(Sleeper/Seater/AC/Non-AC),Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:03,08h 27m,18:30,4.1,INR 606,1 Seat available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:01,07h 54m,19:55,3.5,INR 945,20 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,07h 39m,21:25,2.0,INR 1099,35 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,08h 35m,22:50,4.2,INR 945,39 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,09h 01m,00:01,3.9,INR 801,25 Seats available
...,...,...,...,...,...,...,...,...,...,...
856,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,A1 Travels,A/C Seater / Sleeper (2+1),23:59,07h 01m,07:00,2.2,INR 700,24 Seats available
857,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,EMERALD TRAVELS,A/C Sleeper (2+1),23:00,05h 30m,04:30,3.2,INR 1100,13 Seats available
858,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Jacobs Travels,A/C Semi Sleeper (2+2),18:30,08h 00m,02:30,2.9,INR 950,34 Seats available
859,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,AURA CONNECT,A/C Sleeper (2+1),18:15,08h 05m,02:20,2.7,1899,30 Seats available


In [7]:
import mysql.connector
import pandas as pd

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456",
    autocommit=True,
    use_pure=True # Replace with your database name
)

cursor = conn.cursor()

# Create a new database (if you don't have one yet)
cursor.execute("CREATE DATABASE IF NOT EXISTS myprojdatabase")
cursor.execute("USE myprojdatabase")

print("Connected to MySQL and database created!")

create_table_query="""
CREATE TABLE IF NOT EXISTS Redbus (
    Id INT AUTO_INCREMENT PRIMARY KEY,
    bus_routes_name VARCHAR(255),
    bus_routes_link VARCHAR(255),
    bus_name VARCHAR(255),
    bus_type VARCHAR(100),
    departing_time VARCHAR(50),
    duration VARCHAR(50),
    reaching_time VARCHAR(50),
    star_rating VARCHAR(10),
    price VARCHAR(50),
    seat_availability VARCHAR(50)
);
"""

cursor.execute(create_table_query)
conn.commit()

# SQL Insert query
insert_query = """
INSERT INTO Redbus (
    bus_routes_name, bus_routes_link, bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seat_availability) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for index, row in df.iterrows():
    cursor.execute(insert_query, (
        row['Bus_Routes_Name'], 
        row['Bus_Routes_link'], 
        row['Bus_Name'], 
        row['Bus_Type_(Sleeper/Seater/AC/Non-AC)'], 
        row['Departing_Time'], 
        row['Duration'], 
        row['Reaching_Time'], 
        row['Star_Rating'], 
        row['Price'], 
        row['Seat_Availability']
    ))

# Commit the transaction after inserting all rows
conn.commit()

# Close the connection
cursor.close()
conn.close()

print(f"{len(df)} rows inserted into MySQL database successfully!")



Connected to MySQL and database created!
861 rows inserted into MySQL database successfully!
